# Semantic Segmentation

In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

print("All loaded.")

All loaded.


In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), \
'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


In [3]:
# Download VGG model that is pretrained.
helper.maybe_download_pretrained_vgg('./data')

Done.


In [4]:
# Load pretrained model and test.
def load_vgg(sess, vgg_path):
    tf.saved_model.loader.load(sess, ['vgg16'], vgg_path)
    print("Model loaded.")
    input_tensor = sess.graph.get_tensor_by_name('image_input:0')
    keep_prob_tensor = sess.graph.get_tensor_by_name('keep_prob:0')
    layer3_out_tensor = sess.graph.get_tensor_by_name('layer3_out:0')
    layer4_out_tensor = sess.graph.get_tensor_by_name('layer4_out:0')
    layer7_out_tensor = sess.graph.get_tensor_by_name('layer7_out:0')
    
    return input_tensor, keep_prob_tensor, layer3_out_tensor, layer4_out_tensor, layer7_out_tensor

print("Start test.")
tests.test_load_vgg(load_vgg, tf)

Start test.
Model loaded.
Tests Passed
